In [ ]:
import numpy as np
from PIL import Image
from io import BytesIO
import json
import base64
from encrusted import encode_mask, decode_mask

In [ ]:
def img_bytes_to_b64_str(img_bytes):
    img_bytes_b64 = base64.b64encode(img_bytes)
    img_str_b64 = img_bytes_b64.decode("ascii")
    return img_str_b64

def b64_str_to_img_bytes(b64_str):
    img_bytes_b64 = b64_str.encode("ascii")
    img_bytes = base64.b64decode(img_bytes_b64)
    return img_bytes

In [ ]:
TEST_SIZE = (28, 1532, 1532)

In [ ]:
# make quantized mask
arr = np.random.uniform(0, 1, TEST_SIZE)
quantized_arr = (arr * 256 // 1).astype(np.uint8)

In [ ]:
%%timeit
pngs = []
for mask in quantized_arr:
    img = Image.fromarray(mask)
    bio = BytesIO()
    img.save(bio, "PNG")
    pngs.append(
        {
            "mask":img_bytes_to_b64_str(bio.getvalue())
        }
    )

In [ ]:
# see size
pngs = []
for mask in quantized_arr:
    img = Image.fromarray(mask)
    bio = BytesIO()
    img.save(bio, "PNG")
    pngs.append(
        {
            "mask":img_bytes_to_b64_str(bio.getvalue())
        }
    )

print([len(d["mask"])//1_000 for d in pngs])

In [ ]:
%%timeit
vecs = []
for mask in quantized_arr:
    vecs.append({
        "mask": img_bytes_to_b64_str(encode_mask(mask))
    })

In [ ]:
# see sizes
vecs = []
for mask in quantized_arr:
    vecs.append({
        "mask": img_bytes_to_b64_str(encode_mask(mask))
    })

print([len(d["mask"])//1_000 for d in vecs])

In [ ]:
# assert lossless
for mask in quantized_arr:
    img_str = img_bytes_to_b64_str(encode_mask(mask))
    decoded_mask = decode_mask(b64_str_to_img_bytes(img_str))
    assert (decoded_mask == mask).all()